In [112]:
import pandas as pd
import numpy as np
df = pd.read_csv("Entrenamieto_ECI_2020.csv")
filtered = df
# 3509 casos estariamos perdiendo si no hacemos nada.
filtered['Convertion_Factor'] = filtered['ASP_(converted)'] / filtered['ASP']
filtered.Convertion_Factor.isnull().sum()

3509

In [113]:
# Intentamos recuperar algunos datos del factor de conversion.
filtered['Convertion_Factor_AVG'] = filtered.groupby('Opportunity_ID')['Convertion_Factor'].transform('mean')
filtered['Convertion_Factor_diff_from_AVG'] = np.abs((filtered.Convertion_Factor - filtered.Convertion_Factor_AVG) / filtered.Convertion_Factor_AVG) * 100
# Aca podemos ver que el error maximo ocasionado por el reemplazo de la media del grupo no supera el 3.5%.
filtered['Convertion_Factor_diff_from_AVG'].max()

0.03472863505806406

In [114]:
# La media y la mediana son muy pequeñas
mean_ = filtered.Convertion_Factor_diff_from_AVG.mean()
median_ = filtered.Convertion_Factor_diff_from_AVG.median()
print("Mean: {}".format(mean_))
print("Median: {}".format(median_))

Mean: 5.214521564593748e-05
Median: 0.0


In [115]:
# Redujimos 10 veces la cantidad de nulos
filtered.Convertion_Factor_AVG.isnull().sum()

306

In [116]:
# Eliminamos los que no pudimos salvar
filtered = filtered.loc[filtered.Convertion_Factor_AVG.isnull() == 0]
filtered.Convertion_Factor_AVG.isnull().sum()

0

In [117]:
# Eliminamos los infinitos

filtered = filtered.loc[filtered.Convertion_Factor_AVG != np.inf]
filtered.loc[filtered.Convertion_Factor_AVG == np.inf].shape

(0, 55)

In [118]:
# Ahora veamos lo mismo con Total Amount
# Tenemos 53 valores perdidos, no creo que tenga sentido recuperarlos, ya que modificaria la sumatoria
# del Total Amount del grupo, siendo que son pocos creo que lo mejor es sacarlos.
filtered.Total_Amount.isnull().sum()

53

In [119]:
filtered.shape

(16640, 55)

In [120]:
# Eliminamos todos los 'Opportunity_ID' que tienen algun registro con Total_Amount == None 
filtered['Total_Amount_is_null'] = filtered.Total_Amount.isnull().astype(int)
filtered['Total_Amount_is_null'] = filtered.groupby('Opportunity_ID')['Total_Amount_is_null'].transform('sum')
# Perdimos 359 filas
filtered = filtered.loc[filtered.Total_Amount_is_null == 0]
filtered.shape

(16587, 56)

In [121]:
filtered['Total_Amount(USD)'] = filtered['Total_Amount'] * filtered['Convertion_Factor_AVG']
#Ya que estamos convertimos el Total_Taxable_Amount
filtered['Total_Taxable_Amount(USD)'] = filtered['Total_Taxable_Amount'] * filtered['Convertion_Factor_AVG']
filtered.head(3)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Convertion_Factor,Convertion_Factor_AVG,Convertion_Factor_diff_from_AVG,Total_Amount_is_null,Total_Amount(USD),Total_Taxable_Amount(USD)
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,EUR,5272800.0,Closed Lost,Prod_Category_A_None,1.131096,1.131096,0.0,0,5964043.80,5964043.80
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,EUR,48230.0,Closed Won,Prod_Category_A_None,1.131094,1.131094,0.0,0,54552.68,54552.68
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,USD,83865.6,Closed Won,Prod_Category_A_None,1.000000,1.000000,0.0,0,83865.60,83865.60


In [122]:
filtered['Total_Amount(USD)'].isnull().sum()

0

In [123]:
filtered['Total_Taxable_Amount(USD)'].isnull().sum()

0

In [124]:
filtered.loc[filtered['Total_Amount(USD)'] == np.inf].shape

(0, 58)

In [125]:
filtered.loc[filtered['Total_Taxable_Amount(USD)'] == np.inf].shape

(0, 58)

In [126]:
# Guardamos el dataframe
filtered = filtered.drop(columns={'Total_Amount_is_null', 'Convertion_Factor_AVG', 'Convertion_Factor_diff_from_AVG', 'Convertion_Factor', 'ASP', 'ASP_(converted)', 'ASP_Currency', 'ASP_(converted)_Currency', 'Total_Amount', 'Total_Taxable_Amount', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Price', 'Currency'})
filtered.to_csv('Entrenamieto_ECI_2020_Total_Amount.csv')